# Usage example: Multiverse Analysis

To conduct a multiverse analysis, the forking paths must be specified in a dictionary. Options can contain:

* strings
* numerical values
* boolean values
* comet dFC methods
* comet and bct graph measures
* any kind of function

In [1]:
from comet.multiverse import Multiverse

forking_paths = {
    "strings": ["Hello", "world"],
    "numbers": [1, 2, 4.2],
    "booleans": [True, False],

    "dfc_measures": [{
                        "name": "LeiDA",
                        "func": "comet.connectivity.LeiDA",
                        "args": {
                                "time_series": "ts"
                                }
                        },
                        {
                        "name": "JC11",
                        "func": "comet.connectivity.Jackknife",
                        "args": {
                                "time_series": "ts",
                                "windowsize": 11,
                                }
                        }],

    "graph_measures": [{
                        "name": "efficiency",
                        "func": "comet.graph.efficiency",
                        "args": {
                                "W": "W",
                                "local": True,
                                }
                        },
                        {
                        "name": "clustering",
                        "func": "comet.graph.clustering_coef",
                        "args": {
                                "W": "W",
                                }
                        }]
}

config = {
    "order": [
        ("strings", "numbers", "booleans", "dfc_measures", "graph_measures"),
        ("strings", "booleans", "numbers", "dfc_measures", "graph_measures"),
    ],
    "invalid_paths": [
        ("Hello", 4.2)
    ]
}

With the decisions and options defined, an analysis template has to be specified. This is similar to a standard analysis pipeline with three additional requirements:

* The template is required to be encapsulated in a dedicated function
* Required imports need to be within the template function
* Decision points need to be specified in double brackets: ```{{decision}}```

In this brief example, the corresponding string, number, and boolean decision will be printed in each universe. Then, connevtivity will be estimated with the corresponding dFC method, and local efficiency is calculated with either the BCT or Comet implementation:

In [2]:
def analysis_template():
    import os
    import numpy as np
    import bct
    import comet

    print(f"Decision 1: {{strings}}")
    print(f"Decision 2: {{numbers}}")
    print(f"Decision 3:{{booleans}}")

    # Load example data and calculate dFC + local efficiency
    ts = comet.utils.load_example()
    dfc = {{dfc_measures}}
    dfc = dfc[0] if isinstance(dfc, tuple) else dfc #required as LeiDA returns multiple outputs

    efficiency = np.zeros((ts.shape[0], dfc.shape[1]))
    for i in range(dfc.shape[2]):
        W = dfc[:, :, i]
        W = np.abs(W)
        efficiency[i] = {{graph_measures}}

    result = {"efficiency": efficiency}
    comet.utils.save_universe_results(result, universe=os.path.abspath(__file__))

The forking paths dictionary defines 5 decision points consisting of 2 options each. Thus, the resulting multiverse will contain 2⁵=32 universes. A ```Multiverse``` object has to be created and can then be used to create, run, summarize, and visualize the multiverse.

* ```multiverse.create()``` will generate Python scripts for all 32 universes. These scripts will be saved in a newly created ```example_multiverse/``` folder
* ```multiverse.summary()``` will print the decisions for every universe. This information is also available as a .csv in the ```example_multiverse/results/``` folder
* ```multiverse.run()``` will either run all or individual universes. If the computational resources allow for it, this can be parallelized by using e.g. ```multiverse.run(parallel=4)```

In [3]:
multiverse = Multiverse(name="example_decisions")
multiverse.create(analysis_template, forking_paths, config)
multiverse.summary();

,Universe,Decision 1,Decision 2,Decision 3,Decision 4,Decision 5
0,Universe_1,Hello,1,True,"{'name': 'LeiDA', 'func': 'comet.connectivity....","{'name': 'efficiency', 'func': 'comet.graph.ef..."
1,Universe_2,Hello,1,True,"{'name': 'LeiDA', 'func': 'comet.connectivity....","{'name': 'clustering', 'func': 'comet.graph.cl..."
2,Universe_3,Hello,1,True,"{'name': 'JC11', 'func': 'comet.connectivity.J...","{'name': 'efficiency', 'func': 'comet.graph.ef..."
3,Universe_4,Hello,1,True,"{'name': 'JC11', 'func': 'comet.connectivity.J...","{'name': 'clustering', 'func': 'comet.graph.cl..."
4,Universe_5,Hello,1,False,"{'name': 'LeiDA', 'func': 'comet.connectivity....","{'name': 'efficiency', 'func': 'comet.graph.ef..."


You can now run individual universes by specifying a number, or run all of them (parallelization is then also supported):

In [4]:
multiverse.run(universe_number=1)

Starting analysis for universe 1...
Running universe_1.py
Decision 1: 'Hello'
Decision 2: 1
Decision 3:True
Calculating LeiDA, please wait...
